In [3]:
import re
import pandas as pd
import os
import itertools
import json
import sklearn
import evaluate

In [5]:
###Dataset###
tsv_directory = '/home/michele.maggini/dataset/CLEF_1C/CLEF_1c'
filename = 'CLEF_1C_English.tsv'
df_path = os.path.join(tsv_directory, filename)
df_gt = pd.read_csv(df_path, sep="\t", engine="python")

cd = '/home/michele.maggini/Llama3/'
filename = 'CLEF1C_English.csv'
path = os.path.join(cd, filename)
df_ans = pd.read_csv(path)

In [6]:
df_gt

,tweet_text,class_label
0,"India's gift of 100,000 COVID-19 vaccines arri...",0
1,Here’s what I’m doing while I wait my turn for...,0
2,"This afternoon, I’m hosting an event with the ...",0
3,"Help shops like mine stay open. Mask up, avoid...",0
4,As part of the ongoing nationwide vaccination ...,0
...,...,...
4535,my flintstones gummies from 15 years ago fight...,0
4536,#COVIDã¼19 Something we all need to be aware ...,1
4537,"Some of the most vulnerable people in Alabama,...",0
4538,It would be great if our first responders coul...,0


In [7]:
for index, row in df_ans.iterrows():
    print(row)

0    Based on the given tweet, I predict that it is...
Name: 0, dtype: object
0    Based on the given tweet, I predict that it is...
Name: 1, dtype: object
0    Based on the given tweet, I predict that it is...
Name: 2, dtype: object
0    Based on the given tweet, I predict that it is...
Name: 3, dtype: object
0    Based on the given tweet, I predict that it is...
Name: 4, dtype: object
0    Based on the given tweet, I predict that it is...
Name: 5, dtype: object
0    Based on the given tweet, I predict that it is...
Name: 6, dtype: object
0    0\n\nReasoning:\n\n* Identification of keyword...
Name: 7, dtype: object
0    0\n\nReasoning:\n\n* Identification of keyword...
Name: 8, dtype: object
0    Based on the given tweet, I predict that it is...
Name: 9, dtype: object
0    0\n\nReasoning:\n\n* Identification of keyword...
Name: 10, dtype: object
0    Based on the given tweet, I predict that it is...
Name: 11, dtype: object
0    Based on the given tweet, I predict that it is...
Name: 1

In [8]:
def extract_last_line(text):
    # Split the text into lines
    lines = text.strip().split('\n')
    # Return the last line
    return lines[-1]

In [9]:
listLastline = df_ans['0'].apply(lambda x: extract_last_line(x))
df_lastline = listLastline.to_list()
len(df_lastline)

4540

In [10]:
def remove_keywords(input_list):
    # Define the keywords to be removed
    keywords = ["(True)", "(true)", "(False)", "(false)"]
    
    # Function to remove keywords from a single string
    def remove_from_string(s):
        for keyword in keywords:
            s = s.replace(keyword, "")
        return s.strip()  # Remove leading/trailing whitespace
    
    # Apply the function to each element in the list
    output_list = [remove_from_string(item) for item in input_list]
    
    return output_list

In [11]:
output_list = remove_keywords(df_lastline)
len(output_list)
output_list

["Overall, the tweet appears to be a genuine expression of gratitude and appreciation for a positive action taken by the Indian government, and does not contain any harmful content. Therefore, I label this tweet as '0'.",
 'Overall, the tweet is a responsible and informative message that promotes public health and safety, and it does not contain any harmful language or behavior.',
 "Based on these factors, I label this tweet as '0', indicating that it is not harmful to society.",
 'Label: 0',
 "Based on the given tweet, I predict that it is not harmful to society. The tweet appears to be a factual report about the vaccination rollout and the President and First Lady receiving their vaccines, which is a positive and informative message. The language used is neutral and objective, without any harmful intent or language. The tweet does not contain any threats, harassment, hostile language, disinformation, or cyberbullying. Therefore, I label this tweet as '0'.",
 "Therefore, I label this 

In [12]:
def extract_integers(input_list):
    # This will store the extracted integers
    integers = []
    # This will store the index and sentences with more than one integer and those without keywords
    special_cases = {}
    
    # Define a regex pattern to match integers
    integer_pattern = re.compile(r'\b\d+\b')
    
    for index, item in enumerate(input_list):
 # Skip to the next item since this case is handled
        
        # Find all integers in the string item
        found_integers = integer_pattern.findall(item)
        
        if found_integers:
            if len(found_integers) > 1:
                special_cases[index] = item
                integers.append(2)
            # Convert found integers to int
            else:
                found_integers = list(map(int, found_integers))
           
            # Add the found integers to the result list
                integers.extend(found_integers)
            
            # Check if there are more than one integer and store the element if true

        else:
            # Check for keywords if no integers are found
            item_lower = item.lower()
            if 'true' in item_lower or 'accurate' in item_lower or 'factual':
                integers.append(0)
            elif 'false' in item_lower:
                integers.append(1)
            else:
                # Store the index and sentence if neither integers nor keywords are found
                integers.append(2)
                #special_cases[index] = item
    
    return integers, special_cases

In [13]:
labels_parsed, special_cases = extract_integers(output_list)
len(labels_parsed), len(special_cases)

(4540, 300)

In [14]:
special_cases

{9: "Overall, the tweet appears to be a responsible and informative message that aims to educate and inform the target audience about COVID-19 vaccines. Therefore, I label this tweet as '0', indicating that it is not harmful to society.",
 34: "Overall, the tweet appears to be a harmless and informative message encouraging people to support the COVID-19 vaccines, regardless of their religious beliefs. Therefore, I label it as '0'.",
 37: "Overall, the tweet appears to be a positive and informative message about the military's efforts in combating COVID-19, and therefore, I label it as '0', indicating that it is not harmful to society.",
 38: "Overall, the tweet appears to be a factual and informative post that aims to educate the public about COVID-19 vaccines, and therefore, I label it as '0', indicating that it is not harmful to society.",
 61: 'This tweet appears to be a neutral and informative announcement about the expansion of COVID-19 vaccination services at CVS pharmacies in Fl

In [15]:
labels_parsed

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 2,
 19,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 19,
 0,
 0,
 19,
 2,
 0,
 19,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 65,
 0,
 0,
 19,
 2,
 2,
 19,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 19,
 0,
 0,
 19,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 19,
 0,
 19,
 0,
 0,
 0,
 2,
 0,
 19,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 2,
 19,
 0,
 0,
 1,
 19,
 19,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,


In [16]:
df_gt

,tweet_text,class_label
0,"India's gift of 100,000 COVID-19 vaccines arri...",0
1,Here’s what I’m doing while I wait my turn for...,0
2,"This afternoon, I’m hosting an event with the ...",0
3,"Help shops like mine stay open. Mask up, avoid...",0
4,As part of the ongoing nationwide vaccination ...,0
...,...,...
4535,my flintstones gummies from 15 years ago fight...,0
4536,#COVIDã¼19 Something we all need to be aware ...,1
4537,"Some of the most vulnerable people in Alabama,...",0
4538,It would be great if our first responders coul...,0


In [17]:

gold_ans = df_gt['class_label'].tolist()
print("gold_ans:", gold_ans)


gold_ans: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
k = [el for el in labels_parsed]
print(k)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 19, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 19, 0, 0, 0, 0, 0, 19, 0, 0, 0, 2, 19, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 19, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 1, 0, 19, 0, 0, 19, 2, 0, 19, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 65, 0, 0, 19, 2, 2, 19, 0, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 19, 0, 0, 19, 0, 0, 19, 0, 0, 0, 2, 2, 0, 0, 0, 0, 19, 0, 19, 0, 0, 0, 2, 0, 19, 0, 0, 2, 0, 0, 0, 0, 0, 0, 19, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 2, 19, 0, 0, 1, 19, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 19, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 19, 0, 2, 2, 2, 0, 2, 0, 19, 0, 0, 16, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 19, 0, 0, 0, 19, 0, 0, 19, 0, 19, 19,

In [20]:
def compute_metrics(predictions, labels):
        accuracy_metric = evaluate.load("accuracy")
        precision_metric = evaluate.load("precision")
        recall_metric = evaluate.load("recall")
        f1_metric = evaluate.load("f1")


        accuracy = accuracy_metric.compute(predictions=predictions, references=labels)[
            "accuracy"
        ]
        precision = precision_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["precision"]
        recall = recall_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["recall"]
        f1 = f1_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["f1"]

        return {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1-score": f1,
        }


results=compute_metrics(labels_parsed, gold_ans)
print(f"CLEF1C_English, CoT, prompt with temperature=0.1:\n", json.dumps(results, indent=4))

CLEF1C_English, CoT, prompt with temperature=0.1:
 {
    "accuracy": 0.731057268722467,
    "precision": 0.8710977289593677,
    "recall": 0.731057268722467,
    "f1-score": 0.7922949566584863
}


/home/michele.maggini/miniconda3/envs/bert_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
